### <font color="red"> *Exercise*: starting ipyparallel </font>

Alternative location for configuration files on Windows: `%PROGRAMDATA%\jupyter\`

### <font color="blue"> Demo, direct interface </blue>

In [3]:
import ipyparallel

In [4]:
rc = ipyparallel.Client()

/Users/ktw/anaconda2/lib/python2.7/site-packages/ipyparallel/client/client.py:442: RuntimeWarning: 
            Controller appears to be listening on localhost, but not on this machine.
            If this is true, you should specify Client(...,sshserver='you@130.229.168.241')
            or instruct your controller to listen on an external IP.
  RuntimeWarning)


The `ids` attribute of Client instance shows identifiers of engines that IPython detected

In [5]:
rc.ids

[0, 1, 2, 3]

In [6]:
dview = rc[:]
dview

<DirectView [0, 1, 2, 3]>

In [7]:
dview2 = rc.direct_view()
dview2

<DirectView all>

In [8]:
rc.direct_view?

### <font color="blue"> Demo, %px magic </blue>

In [9]:
%px?

In [10]:
%px import os, time

In [11]:
%px print(time.time())

[stdout:0] 1481548185.94
[stdout:1] 1481548185.94
[stdout:2] 1481548185.95
[stdout:3] 1481548185.95


In [12]:
%px a = os.getpid()

In [14]:
%px print(a)

[stdout:0] 74595
[stdout:1] 74596
[stdout:2] 74598
[stdout:3] 74597


In [15]:
%px print(a)

[stdout:0] 74595
[stdout:1] 74596
[stdout:2] 74598
[stdout:3] 74597


In [16]:
%pxresult

[stdout:0] 74595
[stdout:1] 74596
[stdout:2] 74598
[stdout:3] 74597


In [17]:
%%px --targets :-1
    print(os.getpid())

[stdout:0] 74595
[stdout:1] 74596
[stdout:2] 74598


### <font color="red"> *Exercise: Estimating $\pi$ with parallel Monte Carlo* </font>

In [18]:
import ipyparallel

In [19]:
clients = ipyparallel.Client()
clients.ids

[0, 1, 2, 3]

In [20]:
dview = clients[:]
dview

<DirectView [0, 1, 2, 3]>

In [21]:
import numpy as np
def mc_pi(n):
#    import numpy as np
    count = 0
    p = np.random.rand(n,2)
    for i in range(n):
        x = p[i,0]
        y = p[i,1]
        if (x**2 + y**2) <= 1.0:
            count += 1
    return float(count)/float(n)

In [22]:
%%timeit
est_pi = mc_pi(10**6)
print(est_pi*4) 

3.140236
3.140232
3.143732
3.142344
1 loop, best of 3: 1.25 s per loop


May need to import modules on the engines

In [25]:
dview.execute("import numpy as np")

<AsyncResult: execute>

In [26]:
%%timeit
results = dview.map_sync(mc_pi, [500000]*2)
print(np.sum(results))

1.572378
1.571444
1.570164
1.570518
1 loop, best of 3: 650 ms per loop


Speed up of factor 2 on my mac, since only two physical cores (with hyperthreading enabled)